In [22]:
import mido
import utils
import midiFile

In [23]:
midi_file_path = 'midis/TLS-0.mid'
theMidiFile = midiFile.MidiFile(midi_file_path)

In [24]:
theMidiFile.measures

[[MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('key_signature', key='C', time=0),
  MetaMessage('set_tempo', tempo=631577, time=0),
  MetaMessage('track_name', name='Greensleeves', time=0),
  MetaMessage('text', text='Traditional', time=0),
  MetaMessage('copyright', text='Jim Paterson', time=0),
  Message('program_change', channel=0, program=0, time=0),
  Message('control_change', channel=0, control=121, value=0, time=0),
  Message('control_change', channel=0, control=64, value=0, time=0),
  Message('control_change', channel=0, control=10, value=63, time=0),
  Message('control_change', channel=0, control=7, value=95, time=0),
  Message('note_on', channel=0, note=60, velocity=77, time=0),
  Message('note_on', channel=0, note=52, velocity=63, time=0),
  Message('control_change', channel=0, control=121, value=0, time=0),
  Message('control_change', channel=0, control=64, value=0, time=0),
  Message('c

In [25]:
# Now you have a list of measures, each containing four beats
print(f"Total number of measures: {len(theMidiFile.measures)}")
for i, measure in enumerate(theMidiFile.measures):
    print(f"Measure {i+1}")
    for j, beat in enumerate(measure):
        if beat.type == 'note_on' or beat.type == 'note_off':
            print(f"{j}, {beat} MIDI messages")

Total number of measures: 24
Measure 1
11, note_on channel=0 note=60 velocity=77 time=0 MIDI messages
12, note_on channel=0 note=52 velocity=63 time=0 MIDI messages
18, note_on channel=0 note=48 velocity=64 time=0 MIDI messages
19, note_off channel=0 note=60 velocity=0 time=0.6019718281249999 MIDI messages
Measure 2
0, note_off channel=0 note=52 velocity=0 time=0.56989955859375 MIDI messages
1, note_off channel=0 note=48 velocity=0 time=0 MIDI messages
2, note_off channel=0 note=60 velocity=0 time=0.032072269531249996 MIDI messages
3, note_on channel=0 note=67 velocity=90 time=0.029605171874999996 MIDI messages
4, note_on channel=0 note=60 velocity=77 time=0 MIDI messages
5, note_on channel=0 note=48 velocity=71 time=0 MIDI messages
Measure 3
0, note_on channel=0 note=67 velocity=87 time=0.029605171874999996 MIDI messages
1, note_off channel=0 note=60 velocity=0 time=0.56989955859375 MIDI messages
2, note_off channel=0 note=48 velocity=0 time=0 MIDI messages
3, note_off channel=0 note=

### Save each measure as a separate MIDI file:
Finally, you can save each measure into a separate MIDI file.

In [26]:
# Save the measures.mid
mid_count = 0
for i, measure in enumerate(theMidiFile.measures):
    new_mid = mido.MidiFile()
    new_track = mido.MidiTrack()
    new_mid.tracks.append(new_track)
    new_track.extend(measure)
    mid_count += 1
    new_mid.save(f'measure_{i+1}.mid')

ValueError: message time must be int in MIDI file

In [19]:
# Convert the measures.mid to csv files
for i in range(1, mid_count+1):
    midi_path = f'midis/TLS-0_measures/measure_{i}.mid'
    save_path = f'csvs/TLS-0_measures/measure_{i}.csv'
    utils.midi_to_csv(midi_path, save_path)